## Select Top 2000 Features with ChiSquareSelector and PySpark API

Importing all used libraries.

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, IDF, StopWordsRemover, CountVectorizer, ChiSqSelector, HashingTF
from pyspark.ml.feature import StringIndexer, Normalizer
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

The next static variables can be used to set the input file, a file with stopwords and the output path.

In [2]:
REVIEW_INPUT_DATA_PATH = "hdfs:///user/revies_set.json"
STOP_WORDS_PATH = "stopwords.txt"

I create a new `SparkSession` and read the input data from the provided `REVIEW_INPUT_DATA_PATH` usually locaded at the hdfs.

In [3]:
spark = SparkSession.builder.appName('spark_ml_pipeline').getOrCreate()
df = spark.read.format("json").load(REVIEW_INPUT_DATA_PATH)

In [4]:
(train, test) = df.randomSplit([0.1, 0.1], seed=42)
df= train

Next, I will read the file including the stopwords, remove the `\n` at the end of the lines and write it into a list.

In [5]:
stop_words = []
with open(STOP_WORDS_PATH, encoding='utf-8') as f:
    stop_words = [line.rstrip() for line in f.readlines()]

I just need two columns from the loaded data frame: `category` and `reviewText`.

In [6]:
df_select = df.select("category", "reviewText")

Split the data into train and test data in a ratio of 80%-20%. This is nessacarry at this point, because train and test data need to run trought the entiere pipline.

In [7]:
(train, test) = df_select.randomSplit([0.8, 0.2], seed=42)

The next step includes several parts of the pipeline:

The first part of the pipline is identical to the one in part 2.

`tokenizer` is a `Regextokenizer` that uses the tokens provided to in the Exercise_2.html file to split the input texts into words. The `minTokenLength` is set to `1`.

`str_indexer` is a `StringIndexer` used to convert categories into indices as this is required for the `ChiSquareSelector`. This is just applied on the category column and returns the column `categoryIndex`.

`remover` is a `StopWordRemover` that takes the previously loaded stopwords and removes them from the senteces. It returns the column `filtered`.

`count_vectorizer` is a `CountVectorizer` to convert the input senteces into vectors. I choose the `Countverctorizer` over the `HashingTF` as it is (in my opinion) easier to convert the words back into a human readable representation. It returns the column `rawFeatures`.

`idf` is a `IDF` to compute the inverted document frequence. It returns the column `features`.

`selector` is a `ChiSquareSelector`. This is finally used to select the most descriminative features. It takes the converted `categoryIndex` and the column `features` to select the top 2000 fetures which is specified by `numTopFeatures=2000`.

The following part is additionally to the pipeline in part 2:

`narmalizer` is a `Normalizer`. Takes the input from `selector` and computes normalized vectors. Returns the column `normFeatures`

`svc` is a `LiniearSVC`. It is used to create multiple SVC as part of the one versus rest classifier.

`one_vs_rest` is a `OneVsRest`. I takes the `svc` to build a multiClassifierModel.

In [8]:
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="[0-9\(\)\[\]\{\}\.\!\?\,\;\:\+\=\-\_\"\'\`~#@&*%€$§\\\/\t ]", minTokenLength=1)

str_indexer = StringIndexer(inputCol="category", outputCol="label")

remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=stop_words)

count_vectorizer = CountVectorizer(inputCol="filtered", outputCol="rawFeatures")

idf = IDF(inputCol="rawFeatures", outputCol="features")

selector = ChiSqSelector(numTopFeatures=2000, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="label")

normalizer =  Normalizer(inputCol="selectedFeatures", outputCol="normFeatures")

svc = LinearSVC(maxIter=10, regParam=0.1)

one_vs_rest = OneVsRest( classifier=svc)

Here, I create the pipeline in the correct order. Finally, I fitted and and transformed the data by passing the `df_select` data frame.

In [9]:
pipeline = Pipeline(stages=[tokenizer,str_indexer, remover, count_vectorizer, idf, selector, normalizer, one_vs_rest])

Use a `ParamGridBuilder` to build parameter for grid search (maxIter, regParam, standardization).
A `MulticlassClassificationEvaluator` is used for evaluation with F1 metric.

Finally, I evaluated the model using a `CrossValidator` with the pre-defined  `MulticlassClassificationEvaluator` and `ParamGridBuilder` with 6 foldes. 

In [11]:
param_grid = ParamGridBuilder() \
    .addGrid(svc.maxIter, [5,15])    \
    .addGrid(svc.regParam, [0.1, 0.4, 0.7]) \
    .addGrid(svc.standardization, [True, False]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=6
                          ) 

model = crossval.fit(train)
predictions = model.transform(test)

evaluator.evaluate(predictions)

Py4JJavaError: An error occurred while calling o97.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 386.0 failed 4 times, most recent failure: Lost task 0.3 in stage 386.0 (TID 773, c107.local, executor 5): java.io.IOException: Cannot run program "/home/anaconda3/bin/python3.6": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:197)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:122)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:109)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:77)
	at org.apache.spark.sql.execution.pytho